In [4]:
# import json
# import random

# bbox_path = "/trunk/home/yifanzhu/get_cutouts/IoU_outputs.json"

# # Read the JSON file
# with open(bbox_path, 'r') as f:
#     data = json.load(f)

# # Get all image paths
# image_paths = []
# for item in data:
#     pid = item['PID']
#     image_path = f"P{int(pid):06}.jpg"
#     image_paths.append(image_path)

# # Shuffle the image paths randomly
# random.shuffle(image_paths)

# # Split into training and validation sets with a 9:1 ratio
# split_index = int(len(image_paths) * 0.9)
# train_images = image_paths[:split_index]
# val_images = image_paths[split_index:]

# # Write the paths to the corresponding files
# train_list_path = '/trunk2/yufei/summer24/Cuneiform/Cunei_train_list.txt'
# val_list_path = '/trunk2/yufei/summer24/Cuneiform/Cunei_val_list.txt'

# with open(train_list_path, 'w') as train_file:
#     for path in train_images:
#         train_file.write(f"{path}\n")

# with open(val_list_path, 'w') as val_file:
#     for path in val_images:
#         val_file.write(f"{path}\n")

# print(f"Number of training images: {len(train_images)}")
# print(f"Number of validation images: {len(val_images)}")


Number of training images: 12118
Number of validation images: 1347


In [5]:
# # Define the paths for input and output files
# train_list_path = '/trunk2/yufei/summer24/Cuneiform/Cunei_train_list.txt'
# output_1k_path = '/trunk2/yufei/summer24/Cuneiform/Cunei_train_list_1k.txt'
# output_5k_path = '/trunk2/yufei/summer24/Cuneiform/Cunei_train_list_5k.txt'

# # Read all lines (image paths) from the original train list
# with open(train_list_path, 'r') as f:
#     all_images = f.readlines()

# # Extract the first 1,000 and 5,000 image paths
# images_1k = all_images[:1000]
# images_5k = all_images[1000:6000]

# # Write the 1,000 image paths to the new file
# with open(output_1k_path, 'w') as f_1k:
#     f_1k.writelines(images_1k)

# # Write the 5,000 image paths to the new file
# with open(output_5k_path, 'w') as f_5k:
#     f_5k.writelines(images_5k)

# print(f"1,000 image paths saved to: {output_1k_path}")
# print(f"5,000 image paths saved to: {output_5k_path}")


1,000 image paths saved to: /trunk2/yufei/summer24/Cuneiform/Cunei_train_list_1k.txt
5,000 image paths saved to: /trunk2/yufei/summer24/Cuneiform/Cunei_train_list_5k.txt


In [25]:
import shutil
import os
import tqdm

img_base_path = "/trunk/shared/cuneiform/full_data/images/"

# Define the paths for the source lists and destination directories
train_list_1k_path = '/trunk2/yufei/summer24/Cuneiform/Cunei_train_list_5k.txt'
val_list_path = '/trunk2/yufei/summer24/Cuneiform/Cunei_val_list.txt'

train_dest_dir = '/trunk3/shared/yufei/dataset/Cuneiform/pictures/train'
val_dest_dir = '/trunk3/shared/yufei/dataset/Cuneiform/pictures/val'

# Ensure the destination directories exist
os.makedirs(train_dest_dir, exist_ok=True)
os.makedirs(val_dest_dir, exist_ok=True)

# Function to copy images based on a list
def copy_images(source_list, dest_dir):
    with open(source_list, 'r') as file:
        image_paths = file.readlines()

    for image_path in image_paths:
        image_path = image_path.strip()  # Remove any leading/trailing whitespace
        image_path = os.path.join(img_base_path, image_path)
        if os.path.exists(image_path):  # Check if the file exists
            shutil.copy(image_path, dest_dir)  # Copy the image to the destination directory
        else:
            print(f"File not found: {image_path}")

# Copy the 1k training images
copy_images(train_list_1k_path, train_dest_dir)

# Copy the validation images
copy_images(val_list_path, val_dest_dir)

print("Image copying completed.")


Image copying completed.


In [26]:
import json
import os

# Paths to necessary files and directories
iou_outputs_path = '/trunk/home/yifanzhu/get_cutouts/IoU_outputs.json'
train_list_1k_path = '/trunk2/yufei/summer24/Cuneiform/Cunei_train_list_5k.txt'
val_list_path = '/trunk2/yufei/summer24/Cuneiform/Cunei_val_list.txt'

train_jsons_dir = '/trunk3/shared/yufei/dataset/Cuneiform/jsons/train'
val_jsons_dir = '/trunk3/shared/yufei/dataset/Cuneiform/jsons/val'

# Ensure the destination directories exist
os.makedirs(train_jsons_dir, exist_ok=True)
os.makedirs(val_jsons_dir, exist_ok=True)

# Load the IoU outputs JSON file
with open(iou_outputs_path, 'r') as f:
    iou_data = json.load(f)

# Create a dictionary mapping PID to major_cutout for quick lookup
pid_to_cutout = {item['PID']: item['major_cutout'] for item in iou_data}

# Function to create and save JSON file for each image
def create_json_file(image_list_path, output_dir):
    with open(image_list_path, 'r') as f:
        image_paths = f.readlines()

    for image_path in image_paths:
        image_path = image_path.strip()
        pid_from_name = os.path.basename(image_path).split('.')[0][1:]  # Extract PID from filename (e.g., P000007 -> 7)
        pid = str(int(pid_from_name))  # Convert the PID to an integer and back to a string to remove leading zeros
        # print(pid_from_name, int(pid_from_name), pid)

        if pid in pid_to_cutout:
            major_cutout = pid_to_cutout[pid]
            # Convert the bounding box (x0, y0, x1, y1) to a polygon with 4 corners
            polygon = [
                [major_cutout[0], major_cutout[1]],  # Top-left
                [major_cutout[2], major_cutout[1]],  # Top-right
                [major_cutout[2], major_cutout[3]],  # Bottom-right
                [major_cutout[0], major_cutout[3]]   # Bottom-left
            ]
            json_content = [{
                "polygon": polygon,
                "confidence": 1,
                "channel": "text_line"
            }]
            # Construct the output JSON filename
            json_filename = os.path.splitext(os.path.basename(image_path))[0] + '.json'
            json_output_path = os.path.join(output_dir, json_filename)
            
            # Write the JSON content to the file
            with open(json_output_path, 'w') as json_file:
                json.dump(json_content, json_file, indent=4)
        else:
            print(f"PID {pid} not found in IoU data.")

# Process training images
create_json_file(train_list_1k_path, train_jsons_dir)

# Process validation images
create_json_file(val_list_path, val_jsons_dir)

print("JSON files created and saved successfully.")


JSON files created and saved successfully.
